In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.NR3 import NR3_function

In [2]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])
#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE13_splitmultiphase.dss'
fn = 'IEEE13_nominalload_vec.dss'
fn = 'IEEE13_capnominalvoltage.dss'
#fn = 'hello.dss'
#fn = '02node_threephase_unbalanced_unchanged.dss' #original
fn = '02node_threephase_unbalanced.dss' #apq = .9, az = .1
#fn = '06node_threephase_unbalanced_edit.dss'

times = np.linspace(0, 2*np.pi, 5)
der = [0, 0, 0, 0, 0]
capacitance = [0, 0, 0, 0, 0]

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")


X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None) #generate the all the matrices

for i in range(len(times)):
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i]) 
    #run NR3 with variations in time, DER, and capacitance
    VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
    INR01[i, :, :] = np.reshape(INR, (3, nline))
    STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
VNR
[[ 1.        +0.00000000e+00j  0.99969569+7.20417417e-04j
   0.99949325+1.17249638e-03j]
 [-0.5       -8.66025404e-01j -0.49933889-8.66488739e-01j
  -0.49895979-8.66509828e-01j]
 [-0.5       +8.66025404e-01j -0.50072362+8.65528123e-01j
  -0.50105701+8.65335943e-01j]]
INR:
[[ 0.22509244-2.97978091e-02j  0.10004044+1.19381298e-04j]
 [-0.1512968 -1.72504960e-01j -0.06671418-1.05838166e-01j]
 [-0.04438921+1.76505895e-01j -0.05011205+8.65440184e-02j]]
STXNR:
[[0.22509244+2.97978091e-02j 0.10001008-4.72740959e-05j]
 [0.22504208+4.47743927e-02j 0.12502056+4.95796948e-03j]
 [0.17505319+4.98107631e-02j 0.09999857-3.87546911e-05j]]
SRXNR:
[[0.22500248+2.99509019e-02j 0.09998988-2.02375041e-06j]
 [0.22502198+4.49585377e-02j 0.1249975 +4.99950038e-03j]
 [0.17499754+4.99605607e-02j 0.09999864-2.71147238e-07j]]
iNR
[[ 0.        +0.j          0.12505961-0.02991901j  0.10007404-0.01989274j]
 [ 0.        +0.j         -0.0

In [3]:
#fn= 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'IEEE13_splitmultiphase.dss'
fn = 'IEEE13_nominalload_vec.dss'
fn = 'IEEE13_capnominalvoltage.dss'
#fn = 'hello.dss'
fn = '02node_threephase_unbalanced_unchanged.dss' #original
fn = '02node_threephase_unbalanced.dss' #aPQ = 0.9 aZ = 0.1
#fn = '06node_threephase_unbalanced.dss' #model = 8
#fn = '06node_threephase_unbalanced_edit.dss' 

VNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR02=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")

for i in range(len(times)):
    network1 = nmf2(fn, times[i])
    #network1 = network_mapper_function('compare__lindist3flow_05node_threephase_unbalanced_oscillation_03.dss')

    nnode = network1.nodes.nnode
    nline = network1.lines.nline

    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    network1.loads.aPQ =0.9*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aI = 0.0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aZ = 0.1*np.ones((3,nnode))*(network1.loads.spu != 0)

    network1.loads.spu = 1*network1.loads.spu

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    VNR02[i, :, :] = np.reshape(VNR, (3, nnode))
    INR02[i, :, :] = np.reshape(INR, (3, nline))
    STXNR02[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR02[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR02[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR02[i, :, :] = np.reshape(sNR, (3, nnode))
    print('VNR\n', np.round(VNR,decimals=6))
    print('INR\n', np.round(INR,decimals=6))
    print('STXNR\n', np.round(STXNR,decimals=6))
    print('SRXNR\n', np.round(SRXNR,decimals=6))
    print('iNR\n', np.round(iNR,decimals=6))
    print('sNR\n', np.round(sNR,decimals=6))
    print('itercount:', itercount)

    print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
    print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
    print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

BASE
Vbase: 2886.7513459481293
Sbase: 1000000.0
Ibase: 346.4101615137754
Zbase: 8.333333333333336

NODES
nnode: 3
nodelist: ['sourcebus', 'a01', 'a02']
phases: ['abc', 'abc', 'abc']
PH:
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
LINES
nline: 2
TXnode: ['sourcebus', 'a01']
RXnode: ['a01', 'a02']
TXnum: [0 1]
RXnum: [1 2]
phases: ['abc', 'abc']
PH:
 [[1 1]
 [1 1]
 [1 1]]
config: ['sub_a01', 'a01_a02']
length: [1. 1.]

NODES + LINES
inlines:
 [[-1  0  1]]
innodes:
 [[-1  0  1]]
outlines:
 [[ 0  1 -1]]
outnodes:
 [[ 1  2 -1]]

CONFIGS
nconf: 2
conflist: ['sub_a01', 'a01_a02']
sub_a01 - FZpl:
 [[0.0018+0.0012j 0.0006+0.0048j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0018+0.0012j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0006+0.0048j 0.0018+0.0012j]]
a01_a02 - FZpl:
 [[0.0018+0.0012j 0.0006+0.0048j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0018+0.0012j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0006+0.0048j 0.0018+0.0012j]]

IMPEDANCE
0 - FZpu:
 [[0.0018+0.0012j 0.0006+0.0048j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0018+0.0012j 0.0006+0.


NODES
nnode: 3
nodelist: ['sourcebus', 'a01', 'a02']
phases: ['abc', 'abc', 'abc']
PH:
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
LINES
nline: 2
TXnode: ['sourcebus', 'a01']
RXnode: ['a01', 'a02']
TXnum: [0 1]
RXnum: [1 2]
phases: ['abc', 'abc']
PH:
 [[1 1]
 [1 1]
 [1 1]]
config: ['sub_a01', 'a01_a02']
length: [1. 1.]

NODES + LINES
inlines:
 [[-1  0  1]]
innodes:
 [[-1  0  1]]
outlines:
 [[ 0  1 -1]]
outnodes:
 [[ 1  2 -1]]

CONFIGS
nconf: 2
conflist: ['sub_a01', 'a01_a02']
sub_a01 - FZpl:
 [[0.0018+0.0012j 0.0006+0.0048j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0018+0.0012j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0006+0.0048j 0.0018+0.0012j]]
a01_a02 - FZpl:
 [[0.0018+0.0012j 0.0006+0.0048j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0018+0.0012j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0006+0.0048j 0.0018+0.0012j]]

IMPEDANCE
0 - FZpu:
 [[0.0018+0.0012j 0.0006+0.0048j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0018+0.0012j 0.0006+0.0048j]
 [0.0006+0.0048j 0.0006+0.0048j 0.0018+0.0012j]]
1 - FZpu:
 [[0.0018+0.0012j 0.0006+0.0048j

In [4]:
#nodelist: ['sourcebus', '646', '650', '632', '670', '671', '680', '633', '634', '645', '692', '675', '684', '611', '652']
VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")

kW_list = np.array([])
kvar_list = np.array([])
#dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
#dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
dss.run_command('Redirect 02node_threephase_unbalanced.dss')
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#dss.run_command('Redirect IEEE13_splitmultiphase.dss')
#dss.run_command('Redirect 06node_threephase_unbalanced_edit.dss')
for k in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k])
    kW_list = np.append(kW_list, dss.Loads.kW())
    kvar_list = np.append(kvar_list, dss.Loads.kvar())

for i in range(len(times)):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
    Vbase = dss.Bus.kVBase() * 1000
    Sbase = 1000000.0
    Ibase = Sbase/Vbase
    Zbase = Vbase/Ibase
    
    # Set slack bus (sourcebus) voltage reference in p.u.
    SlackBusVoltage = 1.000
    dss.Vsources.PU(SlackBusVoltage)
    
    kWcurr = dss.Loads.kW()
    kvarcurr = dss.Loads.kvar()
    
    #Time varying load
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        
        dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])) )
        dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
      
    dss.Solution.Convergence(0.000000000001)
    
    # Solve power flow with OpenDSS file
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Initial Solution Not Converged. Check Model for Convergence')
    else:
        print('Initial Model Converged. Proceeding to Next Step.')
        #Doing this solve command is required for GridPV, that is why the monitors
        #go under a reset process
        dss.Monitors.ResetAll()

        #set solution Params
        #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
        dss.Solution.Mode(1)
        dss.Solution.Number(1)
        dss.Solution.StepSize(1)
        dss.Solution.ControlMode(-1)
        dss.Solution.MaxControlIterations(1000000)
        dss.Solution.MaxIterations(30000)
        #Easy process to get all names and count of loads, a trick to avoid
        #some more lines of code
        TotalLoads=dss.Loads.Count()
        AllLoadNames=dss.Loads.AllNames()
        print('OpenDSS Model Compliation Done.')
        print('Iterations: ', dss.Solution.Iterations())
        print('Tolerance: ', dss.Solution.Convergence())

    print('')
    
    # print(dss.Solution.Converged())

    # Print number of buses, and bus names
    print(len(dss.Circuit.AllBusNames()))
    print(dss.Circuit.AllBusNames())

    # Print number of loads, and load names
    print(len(dss.Loads.AllNames()))
    print(dss.Loads.AllNames())

    print('')

    VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

    for k1 in range(len(dss.Circuit.AllBusNames())):
        
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
    #     print(dss.Circuit.AllBusNames()[k1])
    #     print(dss.Bus.Nodes())

    #     print('puVOTLAGES - LN CARTESIAN')
    #     print(dss.Bus.PuVoltage())

        ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
        

        Vtemp = np.asarray(dss.Bus.PuVoltage())

        Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
        

    #     print(np.asarray(dss.Bus.Nodes(),'int'))

        VDSS[ph,k1] = Vtemp    

    #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


    #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
    #     for k2 in range(len(dss.Bus.Nodes())):
    #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))


    IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
        Imn = np.asarray(dss.CktElement.Currents())/Ibase
    #     print(Imn)
        Imn = Imn[0:int(len(Imn)/2)]
    #     print(Imn)
        Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
    #     print(Imn)
        IDSS[ph,k1] = Imn
    #     print('')

    print('IDSS\n', np.round(IDSS,decimals=6))

    STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
    SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
    #     print(dss.CktElement.BusNames())
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    #     print(ph)
        Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
    #     print(Sk)

    #     print(Sk[0:int(len(Sk)/2)])
    #     print(Sk[int(len(Sk)/2):])

        STXtemp = Sk[0:int(len(Sk)/2)]
        SRXtemp = Sk[int(len(Sk)/2):]

        STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
    #     print(STXtemp)

        SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])
    #     print(SRXtemp)

        STXDSS[ph,k1] = STXtemp
        SRXDSS[ph,k1] = SRXtemp

    VDSS0[i, :, :] = VDSS
    IDSS0[i, :, :] = IDSS
    SRXDSS0[i, :, :] = SRXDSS
    STXDSS0[i, :, :] = STXDSS
    
    
    print('STXDSS\n', np.round(STXDSS,decimals=6))
    print('SRXDSS\n', np.round(SRXDSS,decimals=6))

    print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
    print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
    print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  2
Tolerance:  1e-12

3
['sourcebus', 'a01', 'a02']
6
['load_a01_a_01', 'load_a01_b_01', 'load_a01_c_01', 'load_a02_a_01', 'load_a02_b_01', 'load_a02_c_01']

VDSS
 [[ 0.999999-3.00000e-06j  0.999694+7.17000e-04j  0.999492+1.16900e-03j]
 [-0.500002-8.66023e-01j -0.499341-8.66486e-01j -0.498962-8.66507e-01j]
 [-0.499997+8.66025e-01j -0.500721+8.65528e-01j -0.501054+8.65336e-01j]]
IDSS
 [[ 0.225093-2.97970e-02j  0.100041+1.09000e-04j]
 [-0.151283-1.72513e-01j -0.066709-1.05841e-01j]
 [-0.044405+1.76497e-01j -0.050119+8.65400e-02j]]
STXDSS
 [[0.225092+2.9797e-02j 0.10001 -3.8000e-05j]
 [0.225042+4.4757e-02j 0.125021+4.9510e-03j]
 [0.175053+4.9792e-02j 0.099999-4.7000e-05j]]
SRXDSS
 [[0.225002+2.9961e-02j 0.09999 +1.8000e-05j]
 [0.225022+4.4952e-02j 0.124997+5.0040e-03j]
 [0.174997+4.9953e-02j 0.099999+2.0000e-06j]]
|VDSS|
 [[0.999999 0.999695 0.999493]
 [0.999998 1.00007  0.999899]
 [0.999999 0.99

In [ ]:
#dss - opendss solution
#01 - vectorized solution without network mapper
#02 - nonvectorized solution using OpenDSS and network mapper
for i in range(len(times)):
    print(i)
    print('Complex Voltage difference')
    print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 
    print(np.max(np.abs(VDSS0[i] - VNR02[i])))

    print('Complex Current difference')
    print(np.max(np.abs(INR01[i] - INR02[i])))
    print(np.max(np.abs(IDSS0[i] - INR01[i])))
    print(np.max(np.abs(IDSS0[i] - INR02[i])))

    print('Complex TX Power difference')
    print(np.max(np.abs(STXNR01[i] - STXNR02[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR02[i])))

    print('Complex RX Power difference')
    print(np.max(np.abs(SRXNR01[i] - SRXNR02[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR02[i])))

    print('Voltage Magnitude difference')
    print(np.max(np.abs(np.abs(VNR01[i]) - np.abs(VNR02[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR02[i]))))

    print('Voltage Angle difference')
    print(np.max(np.abs(180/np.pi*np.angle(VNR01[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print("\n")

In [ ]:
print(VNR01[0].shape)
print(VNR02[0].shape)
print(VDSS0[0].shape)

In [ ]:
dss.run_command('Redirect 06node_threephase_unbalanced.dss')
for n in range(len(dss.Capacitors.AllNames())):
    dss.Capacitors.Name(dss.Capacitors.AllNames()[n])
    
cap_dict = {}
cap_ph_dict = {}
for n in range(len(dss.Capacitors.AllNames())):
    dss.Capacitors.Name(dss.Capacitors.AllNames()[n])
    pattern =  r"(\w+)\." 
    m = re.findall(pattern, dss.CktElement.BusNames()[0])
    cap_dict[dss.CktElement.BusNames()[0]] = dss.Capacitors.kvar()    
    load_phases = [0, 0, 0]
    for i in range(1, 4): #if the phase is present, what other phases are
        pattern = r"\.%s" % (str(i))
        m2 = re.findall(pattern, dss.CktElement.BusNames()[0])
        if m2:
            load_phases[i - 1] = 1
    cap_ph_dict[dss.CktElement.BusNames()[0]] = load_phases

for n in cap_ph_dict.keys():
    pattern =  r"(\w+)\."
    cap_bus = re.findall(pattern, n)
    print(cap_bus[0])

# for k in range(len(dss.Circuit.AllNodeNames())):
#    # for n in cap_dict.keys():
#     #print(dss.Circuit.AllNodeNames()[k])
#     for n in cap_dict.keys():
#         if n in dss.Circuit.AllNodeNames()[k]:
#             print('capacitor')
print(cap_dict)
print(cap_ph_dict)
            
    

In [ ]:
# dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
# cap_dict = {}
# cap_ph_dict = {}
# for n in range(len(dss.Capacitors.AllNames())):
#     dss.Capacitors.Name(dss.Capacitors.AllNames()[n])
#     #print(dss.CktElement.BusNames()[0])
#     pattern =  r"(\w+)\."  
#     m = re.findall(pattern, dss.CktElement.BusNames()[0])
#     cap_dict[m[0]] = dss.Capacitors.kvar()    
#     load_phases = [0, 0, 0]
#     for i in range(1, 4): #if the phase is present, what other phases are
#         pattern = r"\.%s" % (str(i))
#         m2 = re.findall(pattern, dss.CktElement.BusNames()[0])
#         if m2:
#             load_phases[i - 1] = 1
#     cap_ph_dict[m[0]] = load_phases
    

# for k2 in range(1, len(dss.Lines.AllNames())):
#     dss.Lines.Name(dss.Lines.AllNames()[k2])
#     #print(len(dss.Lines.AllNames()))
#     lc = dss.Lines.LineCode()
#     dss.LineCodes.Name(lc)
    
#     print(lc)
#     print(dss.LineCodes.Xmatrix())
        
        


In [ ]:
dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
nnode = len(dss.Circuit.AllBusNames())
def load_order_f():
    load_order = {}
    for n in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[n])
        pattern =  r"(\w+)\." 
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        if load_bus[0] not in load_order:
            load_order[load_bus[0]] = 1
        elif load_bus[0] in load_order:
            load_order[load_bus[0]] += 1
            
    print(load_order)
    return load_order
load_order_list = load_order_f()

def load_order_off():
    maxloads = max(load_order_list.values())
    load_order_off_lst = np.zeros((nnode, maxloads), dtype = '<U16') 
    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        pattern =  r"(\w+)\." 
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        for j in range(maxloads):
            if not load_order_off_lst[dss.Circuit.AllBusNames().index(load_bus[0]) - 1][j]:  
                load_order_off_lst[dss.Circuit.AllBusNames().index(load_bus[0]) - 1][j] = load_bus[0]
                break
    return load_order_off_lst
print(load_order_off())
    


def load_values():
    load_ph_arr = np.zeros((nnode, max(load_order_list.values()), 3))
    load_kw_arr = np.zeros((nnode, max(load_order_list.values())))
    load_kvar_arr = np.zeros((nnode, max(load_order_list.values())))
    load_kw_arr_ph = np.zeros((3, nnode))
    load_kvar_arr_ph = np.zeros((3, nnode))
    print(dss.Circuit.AllBusNames())
    
    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        pattern =  r"(\w+)\." 
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        load_ph_arr_temp = [0, 0, 0]
        for i in range(1, 4):
            pattern = r"\.%s" % (str(i))
            load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])       
            if load_ph:
                load_ph_arr_temp[i - 1] = 1
        for j in range(max(load_order_list.values())):   
            idxbs = dss.Circuit.AllBusNames().index(load_bus[0])
            if np.all(load_ph_arr[idxbs, j,:] == [0, 0, 0]):
                load_ph_arr[idxbs, j, :] = load_ph_arr_temp
                load_kw_arr[idxbs, j] = dss.Loads.kW() / sum(load_ph_arr_temp)
                load_kvar_arr[idxbs, j] = dss.Loads.kvar() / sum(load_ph_arr_temp)
                for i in range(len(load_ph_arr_temp)):
                    if load_ph_arr_temp[i] == 1:
                        load_kw_arr_ph[i,idxbs] += dss.Loads.kW() / sum(load_ph_arr_temp)
                        load_kvar_arr_ph[i,idxbs] += dss.Loads.kvar() / sum(load_ph_arr_temp)
                        
                
                break
    return load_ph_arr, load_kw_arr, load_kvar_arr, load_kw_arr_ph, load_kvar_arr_ph
load_ph_arr, load_kw_arr, load_kvar_arr, load_kw_arr_ph, load_kvar_arr_ph = load_values()

print(load_ph_arr)
print("\n")
print(load_kw_arr_ph)
#print(np.sum(load_kw_arr, axis=1))
print("\n")
print(load_kvar_arr_ph)

In [ ]:
dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
def load_order_f(): #dict {bus: no_loads}
    load_order = {}
    for n in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[n])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        if load_bus[0] not in load_order: #if bus DNE in dict
            load_order[load_bus[0]] = 1
        elif load_bus[0] in load_order: #if bus E in dict
            load_order[load_bus[0]] += 1 #add 1 onto count of loads
    return load_order
load_order_list = load_order_f()
print(max(load_order_list.values()))

def load_values():
    load_ph_arr = np.zeros((nnode, max(load_order_list.values()), 3))
    # load_kw_arr = np.zeros((nnode, max(load_order_list.values())))
    # load_kvar_arr = np.zeros((nnode, max(load_order_list.values())))
    load_kw_arr_ph = np.zeros((3, nnode))
    load_kvar_arr_ph = np.zeros((3, nnode))
    loads_apq = np.zeros((3, nnode))
    loads_az = np.zeros((3,nnode))

    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0]) #bus name for load
        print(load_bus[0])
        load_ph_arr_temp = [0, 0, 0] #ph of load
        for k in range(1, 4): #update existing ph of load
            pattern = r"\.%s" % (str(k))
            load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])
            if load_ph:
                load_ph_arr_temp[k - 1] = 1
        print(load_ph_arr_temp)
        for j in range(max(load_order_list.values())):
            print(j)
            idxbs = dss.Circuit.AllBusNames().index(load_bus[0]) #what index is bus at in allbusnames()
            print(load_ph_arr[idxbs, j, :])
            if np.all(load_ph_arr[idxbs, j,:] == [0, 0, 0]): #if row is empty
                print('enters')
                load_ph_arr[idxbs, j, :] = load_ph_arr_temp #place ph array there
                for i in range(len(load_ph_arr_temp)):
                    if load_ph_arr_temp[i] == 1:
                        load_kw_arr_ph[i,idxbs] += (dss.Loads.kW() * 1e3 / Sbase / sum(load_ph_arr_temp))
                        load_kvar_arr_ph[i,idxbs] += (dss.Loads.kvar() * 1e3 / Sbase / sum(load_ph_arr_temp))
                        loads_apq[i,idxbs] = 1.0
                        loads_az[i, idxbs] = 0.0
                print(load_kw_arr_ph)
                break
    #return load_ph_arr, load_kw_arr, load_kvar_arr, load_kw_arr_ph, load_kvar_arr_ph, loads_apq, loads_az
    return load_kw_arr_ph, load_kvar_arr_ph, loads_apq, loads_az
load_values()

In [ ]:
for k2 in range(1, len(dss.Circuit.AllBusNames())):
    print(dss.Circuit.AllBusNames()[k2])

In [ ]:
print(dss.Circuit.AllBusNames())#.index('a01')
max(load_order_list.values())

In [ ]:
#dss.run_command('Redirect 02node_threephase_unbalanced_edit.dss')
#dss.run_command('Redirect ieee.dss')
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#dss.run_command('Redirect IEEE13_splitmultiphase.dss')


for k in range(len(dss.Loads.AllNames())):
    dss.Circuit.SetActiveElement('Load.'+dss.Loads.AllNames()[k])
    print(dss.CktElement.Powers())
print("\n")

for j in range(len(dss.Capacitors.AllNames())):
    dss.Capacitors.Name(dss.Capacitors.AllNames()[j])
    print(dss.CktElement.Name())
    print(dss.Capacitors.kvar())
    print(dss.CktElement.Powers())
print("\n")

for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    Vtemp = np.asarray(dss.Bus.VMagAngle()) / Vbase
    print(Vtemp)

In [ ]:
dss.run_command('Redirect 02node_threephase_unbalanced_edit.dss')
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
def cap_dict():
    cap_dict_kvar = {}
    cap_dict_kv = {}
    cap_ph_dict = {}
    for n in range(len(dss.Capacitors.AllNames())):
        dss.Capacitors.Name(dss.Capacitors.AllNames()[n])     
        pattern =  r"(\w+)\."  #if it is, is the phase present?
        m = re.findall(pattern, dss.CktElement.BusNames()[0])
        cap_dict_kvar[m[0]] = dss.Capacitors.kvar()
        cap_dict_kv[m[0]] = dss.Capacitors.kV()
        load_phases = [0, 0, 0]
        for i in range(1, 4): #if the phase is present, what other phases are
            pattern = r"\.%s" % (str(i))
            m2 = re.findall(pattern, dss.CktElement.BusNames()[0])
            if m2:
                if m[0] not in cap_ph_dict.keys():
                    
                    cap_ph_dict[m[0]] = [0, 0, 0]
                    cap_ph_dict[m[0]][i-1] = 1
                else:
                    cap_ph_dict[m[0]][i-1] = 1
        #cap_ph_dict[m[0]] = load_phases
    return cap_dict_kvar, cap_dict_kv, cap_ph_dict
cap_dict_kvar, cap_dict_kv, cap_ph_dict = cap_dict()
    
cap_dict_kvar, cap_dict_kv, cap_ph_dict = cap_dict()

cappu = np.zeros((3,nnode))


# # Iterate through capacitors in configuration file
for ph in range(0, 3):
    for bus in range(len(dss.Circuit.AllBusNames())):
        if dss.Circuit.AllBusNames()[bus] in cap_dict_kvar.keys():
            if cap_ph_dict[dss.Circuit.AllBusNames()[bus]][ph] == 1:
                cappu[ph, bus] = cap_dict_kvar[dss.Circuit.AllBusNames()[bus]] * 1000 / Sbase / sum(cap_ph_dict[dss.Circuit.AllBusNames()[bus]])
#caps.cappu = np.multiply(caps.cappu,nodes.PH)
